In [ ]:
# Used for obtaining the training data
! python ./preprocessing/download_wordvecs.py --download_dir ./data
! python ./preprocessing/squad_preprocess.py --data_dir ./data


File glove.840B.300d.zip successfully loaded


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\temp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!

glove.840B.300d.zip: 0.00B [00:00, ?B/s]
glove.840B.300d.zip:   0%|          | 8.19k/2.18G [00:02<152:57:20, 3.95kB/s]
glove.840B.300d.zip:   0%|          | 32.8k/2.18G [00:02<32:20:07, 18.7kB/s] 
glove.840B.300d.zip:   0%|          | 81.9k/2.18G [00:02<11:02:54, 54.7kB/s]
glove.840B.300d.zip:   0%|          | 197k/2.18G [00:02<3:59:23, 152kB/s]   
glove.840B.300d.zip:   0%|          | 410k/2.18G [00:02<1:43:15, 351kB/s]
glove.840B.300d.zip:   0%|          | 852k/2.18G [00:02<44:56, 807kB/s]  
glove.840B.300d.zip:   0%|          | 1.72M/2.18G [00:03<20:19, 1.78MB/s]
glove.840B.300d.zip:   0%|          | 2.77M/2.18G [00:03<12:46, 2.84MB/s]
glove.840B.300d.zip:   0%|          | 4.70M/2.18G [00:03<06:31, 5.55MB/s]
glove.840B.300d.zip:   0%|          | 5.08M/2.18G [00:03<06:59, 5.18MB/s]
glove.840B.300d.zip:   0%|          | 7.57M/2

Will download SQuAD datasets to ./data

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\temp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!

train-v1.1.json: 0.00B [00:00, ?B/s]
train-v1.1.json:   0%|          | 8.19k/30.3M [00:02<2:35:56, 3.24kB/s]
train-v1.1.json:   0%|          | 106k/30.3M [00:02<09:06, 55.2kB/s]   
train-v1.1.json:   4%|▍         | 1.17M/30.3M [00:02<00:36, 795kB/s]
train-v1.1.json:  18%|█▊        | 5.35M/30.3M [00:02<00:05, 4.48MB/s]
train-v1.1.json:  38%|███▊      | 11.6M/30.3M [00:02<00:01, 10.9MB/s]
train-v1.1.json:  55%|█████▍    | 16.6M/30.3M [00:03<00:00, 16.1MB/s]
train-v1.1.json:  75%|███████▍  | 22.7M/30.3M [00:03<00:00, 23.1MB/s]
train-v1.1.json:  82%|████████▏ | 24.7M/30.3M [00:03<00:00, 22.5MB/s]
train-v1.1.json:  85%|████████▍ | 25.7M/30.3M [00:03<00:00, 19.4MB/s]
train-v1.1.json:  89%|████████▉ | 26.9M/30.3M [00:03<00:00, 17.7MB/s]
train-v1.1.json:  95%|█████████▌| 28.9M/30.3M [00:03<00:00, 17.9MB/s]
train-v1.1.json:  99%|████████


Will put preprocessed SQuAD datasets in ./data
File train-v1.1.json successfully loaded
Train data has 87599 examples total
Number of (context, question, answer) triples discarded due to char -> token mapping problems:  97
Number of (context, question, answer) triples discarded because character-based answer span is unaligned with tokenization:  1173
Number of (context, question, answer) triples discarded due character span alignment problems (usually Unicode problems):  23
Processed 86306 examples of total 87599

File dev-v1.1.json successfully loaded
Dev data has 10570 examples total
Number of (context, question, answer) triples discarded due to char -> token mapping problems:  0
Number of (context, question, answer) triples discarded because character-based answer span is unaligned with tokenization:  176
Number of (context, question, answer) triples discarded due character span alignment problems (usually Unicode problems):  0
Processed 10394 examples of total 10570



From the paper:
"We use a max sequence length of 600 during training and a hidden state size of 200 for all recurrent
units, maxout layers, and linear layers. All LSTMs have randomly initialized parameters and an
initial state of zero. Sentinel vectors are randomly initialized and optimized during training. For
the dynamic decoder, we set the maximum number of iterations to 4 and use a maxout pool size of
16. We use dropout to regularize our network during training (Srivastava et al., 2014), and optimize
the model using ADAM (Kingma & Ba, 2014). All models are implemented and trained with
Chainer (Tokui et al., 2015)."

In [1]:
from model import DynamicCoattentionNW
import chainer as ch


max_seq_length = 600

hid_state_size = 200

dyn_dec_max_it = 4
maxout_pool_size = 16

dropout = 0.1

c:\Users\temp\anaconda3\envs\DLNLP-Project\lib\site-packages\chainer\_environment_check.py:72: UserWarning: 
--------------------------------------------------------------------------------
CuPy (cupy) version 12.3.0 may not be compatible with this version of Chainer.
Please consider installing the supported version by running:
  $ pip install 'cupy>=7.7.0,<8.0.0'

See the following page for more details:
  https://docs.cupy.dev/en/latest/install.html
--------------------------------------------------------------------------------

  warnings.warn(msg.format(


In [3]:
from preprocessing.vocab import get_glove

glove_path = "./data/glove.840B.300d.txt"
glove_vector_size = 300
emb_mat, word2id, id2word = get_glove(glove_path, glove_vector_size)

vocab_size = len(word2id)

print(emb_mat.shape)

Loading GLoVE vectors from file: ./data/glove.840B.300d.txt


100%|██████████| 2196017/2196017 [01:47<00:00, 20516.69it/s]

(2196018, 300)


In [ ]:
from data_batcher import get_batch_generator
batchsize = 2

train_context_path = "./data/train.context"
train_qn_path = "./data/train.question"
train_ans_path = "./data/train.answer"
train_span_path = "./data/train.span"

batch_gen = get_batch_generator(word2id, train_context_path, train_qn_path, train_span_path, batchsize, max_seq_length, max_seq_length, discard_long=True)

for batch in batch_gen:
    # print(batch.context_ids)
    # print(batch.context_mask)
    # print(batch.context_tokens)

    # print(batch.qn_ids)
    # print(batch.qn_mask)
    # print(batch.qn_tokens)

    # print(batch.ans_span)
    # print(batch.ans_tokens)   
    break

# train_iter = ch.iterators.SerialIterator(train, batchsize)
# test_iter = ch.iterators.SerialIterator(test, batchsize, False, False)

Refilling batches...
Refilling batches took 0.03699445724487305 seconds
1
1


In [ ]:
model = DynamicCoattentionNW(max_seq_length, hid_state_size, dyn_dec_max_it, maxout_pool_size, dropout, emb_mat)

In [ ]:
# Setup an optimizer
optimizer = ch.optimizers.Adam()
optimizer.setup(model)


In [ ]:
# Create the updater, using the optimizer
updater = ch.training.StandardUpdater(train_iter, optimizer, device=-1)

In [ ]:
# Set up a trainer
max_epoch = 10
trainer = ch.training.Trainer(updater, (max_epoch, 'epoch'), out='result')

In [ ]:
#save model
#see https://docs.chainer.org/en/stable/guides/serializers.html